# Part 0: Setup Data Owner 1
First of all we need to establish a connection between the points. Using Duet we will establish an P2P connecton. DS will be able to perform remore execurion on the DOs's data on the DS's machine.

### 0.1 Lauch the server
Firstly we import PySyft and launch the duet. Than we send the code which will be displayed below(assuming that the url is known at the both sides) to the DS to establish the connection. This part can be sckiped if setting the parameter lookback to true on the both sides. 

In [1]:
import syft as sy 
#duet = sy.launch_duet(loopback=True)
duet = sy.launch_duet(network_url="http://ec2-3-15-165-254.us-east-2.compute.amazonaws.com:5000")

#OTHER URLs WHICH CAN BE USED
#"http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000"
#"http://ec2-3-138-111-189.us-east-2.compute.amazonaws.com:5000"
#"http://ec2-3-15-165-254.us-east-2.compute.amazonaws.com:5000"

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-3-15-165-254.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!
♫♫♫ > Duet Server ID: a27c8bc011af31164a3e3d164d4e0190

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.duet("a27c8bc011af31164a3e3d164d4e0190")

♫♫♫ > STEP 2: Ask your partner for their Client ID and enter it below!
♫♫♫ > Duet Partner's Client ID: 62ed5faa7d167002ba8cb23a665dc04d

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



#### Now return to the DS notebook to get and insert Server ID (Jump 0.1). Then come back to insert DS's ID (Jump 0.2).  After that go to the DO2 notebook (Jump 0.3)

# Part 1: Data Prepreparation DO 1

### 1.1 Check the data ( DO1)
Next we import the neccessary libraries. For this example we will use the wine dataset(https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html).

In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import torch as th

We split the data into the features (X) and the target labels (y). Than we convert it into a pandas dataframe in order to display the data

In [3]:
wine = datasets.load_wine()
X1, _, y1, _ = train_test_split(wine.data, wine.target, test_size=0.6, random_state=1)
print(pd.DataFrame(X1, columns=wine.feature_names).mean())
pd.DataFrame(X1, columns=wine.feature_names).head()

alcohol                          12.978310
malic_acid                        2.404789
ash                               2.392113
alcalinity_of_ash                20.146479
magnesium                        97.971831
total_phenols                     2.271831
flavanoids                        1.944789
nonflavanoid_phenols              0.363803
proanthocyanins                   1.447042
color_intensity                   5.127465
hue                               0.967183
od280/od315_of_diluted_wines      2.553239
proline                         720.000000
dtype: float64


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.13,4.10,2.74,24.5,96.0,2.05,0.76,0.56,1.35,9.20,0.61,1.60,560.0
1,13.30,1.72,2.14,17.0,94.0,2.40,2.19,0.27,1.35,3.95,1.02,2.77,1285.0
2,12.08,2.08,1.70,17.5,97.0,2.23,2.17,0.26,1.40,3.30,1.27,2.96,710.0
3,12.85,1.60,2.52,17.8,95.0,2.48,2.37,0.26,1.46,3.93,1.09,3.63,1015.0
4,11.82,1.72,1.88,19.5,86.0,2.50,1.64,0.37,1.42,2.06,0.94,2.44,415.0


### 1.2 Woking with Pysyft ( DO1)
The important part is that PySyft is designed to work with tensors. Fortunately, there are several libraries in python, such as Pytorch, which are very well designed to work with them. So, what we need to do is to convert the data into tensors to work with it.


In [4]:
X1 = th.FloatTensor(X1)
y1 = th.FloatTensor(y1)
print("Length of dataset:", len(X1))

Length of dataset: 71


Even though we try to preserve privacy as much as possible, the DS, obviously, must know what kind of data he is supposed to work with. We will provide some description to the data and the tags, which the DS will use to refer to the data.

In [5]:
X1 = X1.tag("Wine-data")
y1 = y1.tag("Wine-labels")

X1 = X1.describe(
    "Wine quality dataset #1 with 13 features (Alcohol, Malic acid, Ash, Alcalinity of ash, Magnesium, Total phenols, Flavanoids, Nonflavanoid phenols, Proanthocyanins, Color intensity, Hue, OD280/OD315 of diluted wines, Proline)")
y1 = y1.describe("Labels for wine: class_0, class_1, class_2")

Next step is to upload the data to the Duet server and create pointers to the data. So every time the DS will want to use somehow the data, we will recieve a notification which we can accept or reject, meaning that the DS can not use the data without our permission. 

In [6]:
data_pointer1 = X1.send(duet, pointable=True)
target_pointer1 = y1.send(duet, pointable=True)

In [7]:
# Data owner 1 can verify that the object stored in the tensor
duet.store

[<syft.proxy.torch.TensorPointer object at 0x00000228038D6BB0>, <syft.proxy.torch.TensorPointer object at 0x00000228038D6DC0>]

Similarly to the dataframe format in pandas, we display an information about the tensors.

In [8]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 10e9a8184ffa42c8835b0712f8c602cc>,[Wine-data],Wine quality dataset #1 with 13 features (Alco...,<class 'torch.Tensor'>
1,<UID: baca0f2c081b446399cc2d0da441e82e>,[Wine-labels],"Labels for wine: class_0, class_1, class_2",<class 'torch.Tensor'>


[2022-04-22T18:17:26.081681+0200][CRITICAL][logger]][8672] You do not have permission to .get() Object with ID: <UID: 3307ad3bf4304915b29834d293ab044c>Please submit a request.
[2022-04-22T18:17:26.082031+0200][CRITICAL][logger]][8672] You do not have permission to .get() Object with ID: <UID: 3307ad3bf4304915b29834d293ab044c>Please submit a request.


#### Now go and perform the same procedure with DO2 notebook (Jump 1.1)

# Part 2: Working With The Data (DO 1)

If we take a look at a DUET LIVE STATUS we will see that there is a request waiting for our approval. Let us check its content.

In [9]:
duet.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,"[Wine-data, mean, __getitem__]",I NEED AN AVERAGE YOU DONUT!!!,<UID: a8909654058c4f14a250afb49ed2033c>,<UID: 3307ad3bf4304915b29834d293ab044c>,


In [10]:
duet.requests[0].request_description

'I NEED AN AVERAGE YOU DONUT!!!'

Doesn't seem to be the propper way to describe the request. It is better to decline it for good and check if there are any other requests left.

In [11]:
duet.requests[0].deny()
duet.requests.pandas

""


[2022-04-22T18:17:48.016515+0200][CRITICAL][logger]][8672] You do not have permission to .get() Object with ID: <UID: 3307ad3bf4304915b29834d293ab044c>Please submit a request.
[2022-04-22T18:17:48.017561+0200][CRITICAL][logger]][8672] You do not have permission to .get() Object with ID: <UID: 3307ad3bf4304915b29834d293ab044c>Please submit a request.


#### Return to the DS notebook and proceed (Jump 2.2)

# Part 3: Creating and training the models (DO 1)

The DO can add requests to be accepted or denied by adding them to request handlers. On the contrrary, if we don't specify anyhing, than it means we are willing to accept everything

In [12]:
duet.requests.add_handler(action="accept")

#### Get to the DO2 notebook (Jump 3.2)